## Data Preprocessing

In [95]:
import json
import pandas as pd

In [96]:
# Load Data
df = pd.read_csv(r'../data/raw-data.csv')

# Airport Code --> Airport Name / Latitude / Longitude Mapping
# https://github.com/ip2location/ip2location-iata-icao
airports = pd.read_csv(r'../data/iata-icao.csv')
airports = airports[~airports.iata.isnull()] # Remove NULLs
airports = airports[~((airports.iata == 'SGG') & (airports.icao.isnull()))] # Remove duplicate line, IATA = SGG

# Airline Code --> Airline Name Mapping
# https://gist.github.com/AndreiCalazans/390e82a1c3edff852137cb3da813eceb
with open(r'../data/airlines.json', 'r') as f:
    airlines = json.loads(f.read())

# Impute missing airlines
# https://www.iata.org/en/publications/directories/code-search/?airline.search=RP
additional_airline_codes = {
    '3M': 'Silver Airways LLC',
    'NJ': 'Unknown',
    'RP': 'Budapest Aircraft Service Kft.',
    'UK': 'Unknown',
    'TB': 'TUI Airlines Belgium N.V dba TUI fly',
    'YY': 'Unknown',
    '9N': 'Tropic Air Limited'
}

airlines.update(additional_airline_codes)

# Quarter --> Season Mapping
seasons = {
    1: 'Spring',
    2: 'Summer',
    3: 'Fall',
    4: 'Winter'
}

# Airfare dataset has some incorrect, missing, and/or outdated IATA airport codes
# Remap to 
iata_code_remapping = {
    'AIY': 'ACY', # Atlantic City, NJ
    'BHC': 'IFP', # Bullhead City, AZ
    'CHI': 'ORD', # Chicago, IL
    'DET': 'DTW', # Detroit, MI
    'DTT': 'DTW', # Detroit, MI
    'GYY': 'ORD', # Chicago, IL
    'JRB': 'LGA', # New York City, NY (Metropolitan Area)
    'NYC': 'LGA', # New York City, NY (Metropolitan Area)
    'TSS': 'LGA', # New York City, NY (Metropolitan Area)
    'WAS': 'IAD', # Washington, DC (Metropolitan Area)
    'WHR': 'EGE'  # Eagle, CO
}

iata_func = lambda x: iata_code_remapping[x] if x in iata_code_remapping else x

df['airport_1'] = df['airport_1'].map(iata_func)
df['airport_2'] = df['airport_2'].map(iata_func)

/var/folders/90/kz_5bmj53wlfsbx400_tzyr80000gn/T/ipykernel_58489/2674359434.py:2: DtypeWarning: Columns (20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'../data/raw-data.csv')


In [97]:
# Raw Data
with pd.option_context('display.max_columns', 100):
    display(df.head())

,tbl,Year,quarter,citymarketid_1,citymarketid_2,city1,city2,airportid_1,airportid_2,airport_1,airport_2,nsmiles,passengers,fare,carrier_lg,large_ms,fare_lg,carrier_low,lf_ms,fare_low,Geocoded_City1,Geocoded_City2,tbl1apk
0,Table1a,2021,3,30135,33195,"Allentown/Bethlehem/Easton, PA","Tampa, FL (Metropolitan Area)",10135,14112,ABE,PIE,970,180,81.43,G4,1.0000,81.43,G4,1.0000,81.43,NaN,NaN,202131013514112ABEPIE
1,Table1a,2021,3,30135,33195,"Allentown/Bethlehem/Easton, PA","Tampa, FL (Metropolitan Area)",10135,15304,ABE,TPA,970,19,208.93,DL,0.4659,219.98,UA,0.1193,154.11,NaN,NaN,202131013515304ABETPA
2,Table1a,2021,3,30140,30194,"Albuquerque, NM","Dallas/Fort Worth, TX",10140,11259,ABQ,DAL,580,204,184.56,WN,0.9968,184.44,WN,0.9968,184.44,NaN,NaN,202131014011259ABQDAL
3,Table1a,2021,3,30140,30194,"Albuquerque, NM","Dallas/Fort Worth, TX",10140,11298,ABQ,DFW,580,264,182.64,AA,0.9774,183.09,AA,0.9774,183.09,NaN,NaN,202131014011298ABQDFW
4,Table1a,2021,3,30140,30466,"Albuquerque, NM","Phoenix, AZ",10140,14107,ABQ,PHX,328,398,177.11,WN,0.6061,184.49,AA,0.3939,165.77,NaN,NaN,202131014014107ABQPHX


In [98]:
# IATA Codes
airports.head()

,country_code,region_name,iata,icao,airport,latitude,longitude
0,AE,Abu Zaby,AAN,OMAL,Al Ain International Airport,24.2617,55.6092
1,AE,Abu Zaby,AUH,OMAA,Abu Dhabi International Airport,24.4330,54.6511
2,AE,Abu Zaby,AYM,NaN,Yas Island Seaplane Base,24.4670,54.6103
3,AE,Abu Zaby,AZI,OMAD,Al Bateen Executive Airport,24.4283,54.4581
4,AE,Abu Zaby,DHF,OMAM,Al Dhafra Air Base,24.2482,54.5477


In [99]:
# Airline Codes
for i, (k, v) in enumerate(airlines.items()):
    if i < 5:
        print(k, v)

U2 United Feeder Service
1T Bulgarian Air Charter
Q5 40-Mile Air
4O Interjet
7A Express Air Cargo


In [100]:
# Shape prior to joins
df.shape

(245955, 23)

In [117]:
# Joins
output = (
    df
        .merge(airports.rename(columns={'airport': 'airport_name'}).add_suffix('_1'), left_on='airport_1', right_on='iata_1', how='left')
        .merge(airports.rename(columns={'airport': 'airport_name'}).add_suffix('_2'), left_on='airport_2', right_on='iata_2', how='left')
)

# Airline Name
output['carrier_lg_name'] = df['carrier_lg'].map(airlines)
output['carrier_low_name'] = df['carrier_low'].map(airlines)
output['season'] = df['quarter'].map(seasons)

# Fill NAs
output['carrier_lg'] = output['carrier_lg'].fillna('UNK')
output['carrier_lg_name'] = output['carrier_lg_name'].fillna('Unknown')
output['carrier_low'] = output['carrier_low'].fillna('UNK')
output['carrier_low_name'] = output['carrier_low_name'].fillna('Unkown')

# Concat for app display name
output['airport_name_concat_1'] = output['iata_1'] + ' - ' + output['airport_name_1']
output['airport_name_concat_2'] = output['iata_2'] + ' - ' + output['airport_name_2']
output['carrier_lg_name_concat'] = output['carrier_lg'] + ' - ' + output['carrier_lg_name']
output['carrier_low_name_concat'] = output['carrier_low']  + ' - ' + output['carrier_low_name']

In [118]:
# Verify no duplicate rows due to left joins
output.shape

(245955, 44)

In [119]:
# Interrogate
with pd.option_context('display.max_columns', 100):
    display(output.head())

,tbl,Year,quarter,citymarketid_1,citymarketid_2,city1,city2,airportid_1,airportid_2,airport_1,airport_2,nsmiles,passengers,fare,carrier_lg,large_ms,fare_lg,carrier_low,lf_ms,fare_low,Geocoded_City1,Geocoded_City2,tbl1apk,country_code_1,region_name_1,iata_1,icao_1,airport_name_1,latitude_1,longitude_1,country_code_2,region_name_2,iata_2,icao_2,airport_name_2,latitude_2,longitude_2,carrier_lg_name,carrier_low_name,season,airport_name_concat_1,airport_name_concat_2,carrier_lg_name_concat,carrier_low_name_concat
0,Table1a,2021,3,30135,33195,"Allentown/Bethlehem/Easton, PA","Tampa, FL (Metropolitan Area)",10135,14112,ABE,PIE,970,180,81.43,G4,1.0000,81.43,G4,1.0000,81.43,NaN,NaN,202131013514112ABEPIE,US,Pennsylvania,ABE,KABE,Lehigh Valley International Airport,40.6521,-75.4408,US,Florida,PIE,KPIE,St. Pete-Clearwater International Airport,27.9102,-82.6874,Allegiant Air,Allegiant Air,Fall,ABE - Lehigh Valley International Airport,PIE - St. Pete-Clearwater International Airport,G4 - Allegiant Air,G4 - Allegiant Air
1,Table1a,2021,3,30135,33195,"Allentown/Bethlehem/Easton, PA","Tampa, FL (Metropolitan Area)",10135,15304,ABE,TPA,970,19,208.93,DL,0.4659,219.98,UA,0.1193,154.11,NaN,NaN,202131013515304ABETPA,US,Pennsylvania,ABE,KABE,Lehigh Valley International Airport,40.6521,-75.4408,US,Florida,TPA,KTPA,Tampa International Airport,27.9755,-82.5332,Delta Air Lines,United Airlines,Fall,ABE - Lehigh Valley International Airport,TPA - Tampa International Airport,DL - Delta Air Lines,UA - United Airlines
2,Table1a,2021,3,30140,30194,"Albuquerque, NM","Dallas/Fort Worth, TX",10140,11259,ABQ,DAL,580,204,184.56,WN,0.9968,184.44,WN,0.9968,184.44,NaN,NaN,202131014011259ABQDAL,US,New Mexico,ABQ,KABQ,Albuquerque International Sunport,35.0402,-106.6090,US,Texas,DAL,KDAL,Dallas Love Field,32.8471,-96.8518,Southwest Airlines,Southwest Airlines,Fall,ABQ - Albuquerque International Sunport,DAL - Dallas Love Field,WN - Southwest Airlines,WN - Southwest Airlines
3,Table1a,2021,3,30140,30194,"Albuquerque, NM","Dallas/Fort Worth, TX",10140,11298,ABQ,DFW,580,264,182.64,AA,0.9774,183.09,AA,0.9774,183.09,NaN,NaN,202131014011298ABQDFW,US,New Mexico,ABQ,KABQ,Albuquerque International Sunport,35.0402,-106.6090,US,Texas,DFW,KDFW,Dallas/Fort Worth International Airport,32.8968,-97.0380,American Airlines,American Airlines,Fall,ABQ - Albuquerque International Sunport,DFW - Dallas/Fort Worth International Airport,AA - American Airlines,AA - American Airlines
4,Table1a,2021,3,30140,30466,"Albuquerque, NM","Phoenix, AZ",10140,14107,ABQ,PHX,328,398,177.11,WN,0.6061,184.49,AA,0.3939,165.77,NaN,NaN,202131014014107ABQPHX,US,New Mexico,ABQ,KABQ,Albuquerque International Sunport,35.0402,-106.6090,US,Arizona,PHX,KPHX,Phoenix Sky Harbor International Airport,33.4343,-112.0120,Southwest Airlines,American Airlines,Fall,ABQ - Albuquerque International Sunport,PHX - Phoenix Sky Harbor International Airport,WN - Southwest Airlines,AA - American Airlines


In [120]:
# Rename columns
renamed_cols = {
    'Year': 'year',
    'airport_1': 'airport_iata_1',
    'airport_2': 'airport_iata_2',
    'region_name_1': 'state_1',
    'region_name_2': 'state_2'
}

# Reorder columns
cols = [
    'year', 
    'quarter', 
    'season',

    'citymarketid_1', 
    'city1', 
    'airportid_1', 
    'airport_iata_1', 
    'airport_name_1', 
    'airport_name_concat_1', 
    'state_1', 
    'latitude_1', 
    'longitude_1', 

    'citymarketid_2', 
    'city2', 
    'airportid_2', 
    'airport_iata_2', 
    'airport_name_2', 
    'airport_name_concat_2', 
    'state_2', 
    'latitude_2', 
    'longitude_2', 

    'nsmiles', 
    'passengers', 
    'fare', 
    'carrier_lg', 
    'carrier_lg_name', 
    'carrier_lg_name_concat', 
    'large_ms', 
    'fare_lg', 
    'carrier_low', 
    'carrier_low_name', 
    'carrier_low_name_concat', 
    'lf_ms', 
    'fare_low'
]

# Output processed CSV
output = output.rename(columns=renamed_cols)[cols]
output.to_csv(r'../data/processed-data.csv')